In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers, models
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Download the Dataset (15 classes)

In [ ]:

# Load class names from metadata
_, info = tfds.load('food101', split='train', with_info=True)
all_classes = info.features['label'].names

# Define the 15 selected classes
selected_classes = [
    'pizza', 'samosa', 'falafel', 'donuts', 'macaroni_and_cheese',
    'caprese_salad', 'bibimbap', 'ceviche', 'bruschetta', 'beef_carpaccio',
    'gnocchi', 'club_sandwich', 'grilled_cheese_sandwich', 'pad_thai', 'takoyaki'
]
selected_indices = tf.constant([all_classes.index(cls) for cls in selected_classes])

# Filter and remap function
def filter_classes(image, label):
    label = tf.cast(label, tf.int32)
    return tf.reduce_any(tf.equal(label, selected_indices))

def remap_labels(image, label):
    label = tf.cast(label, tf.int32)
    new_label = tf.argmax(tf.cast(tf.equal(label, selected_indices), tf.int32))
    return image, new_label

# Load and filter datasets
training_set = tfds.load('food101', split='train[:90%]', as_supervised=True)
validation_set = tfds.load('food101', split='train[90%:]', as_supervised=True)
testing_set = tfds.load('food101', split='validation', as_supervised=True)

training_set = training_set.filter(filter_classes).map(remap_labels)
validation_set = validation_set.filter(filter_classes).map(remap_labels)
testing_set = testing_set.filter(filter_classes).map(remap_labels)

# Final class names for reference
class_names = selected_classes


# Data Processing

In [ ]:
def preprocess_image(image, label):
    image = tf.image.resize(image, [224, 224]) / 255.0
    return image, label

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, 0.7, 1.3)
    image = tf.image.random_saturation(image, 0.7, 1.3)
    image = tf.image.random_hue(image, 0.05)
    image = tf.image.resize_with_crop_or_pad(image, 230, 230)
    image = tf.image.random_crop(image, [224, 224, 3])
    return image, label

training_set = (training_set
                .map(preprocess_image)
                .map(augment)
                .shuffle(1000)
                .batch(32)
                .prefetch(tf.data.AUTOTUNE))

validation_set = (validation_set
                  .map(preprocess_image)
                  .batch(32)
                  .prefetch(tf.data.AUTOTUNE))

testing_set = (testing_set
               .map(preprocess_image)
               .batch(32)
               .prefetch(tf.data.AUTOTUNE))

In [ ]:
# _, info = tfds.load('food101', split='train', with_info=True)
# class_names = info.features['label'].names[:20]

In [ ]:
batch = next(iter(training_set))
images, labels = batch

images = images.numpy()
labels = labels.numpy()

plt.figure(figsize=(15, 10))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(np.clip(images[i], 0, 1))
    plt.title(class_names[labels[i]])
    plt.axis('off')
plt.show()

# Model Building

In [ ]:
from tensorflow.keras import layers, models, regularizers

model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),

    layers.Conv2D(64, (5, 5), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),

    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),

    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),

    layers.Dense(1024, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Dense(512, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    # Final output layer for 15 classes
    layers.Dense(15, activation='softmax')
])

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)




# Show model summary
model.summary()

# Model Training

In [ ]:
history = model.fit(training_set,
                    epochs=100,
                    validation_data=validation_set)

# Model Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(testing_set)
print(f'Test accuracy: {test_acc:.4f}')

In [ ]:
from sklearn.metrics import classification_report

def evaluate_model(model, testing_set, class_names):
    y_true = []
    y_pred = []

    for images, labels in testing_set:
        preds = model.predict(images)
        y_true.extend(labels.numpy())
        y_pred.extend(tf.argmax(preds, axis=1).numpy())

    report = classification_report(y_true, y_pred, target_names=class_names)
    print(report)


In [ ]:
evaluate_model(model, testing_set, class_names)

In [ ]:
y_true = []
y_pred = []
for images, labels in testing_set:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute and visualize the confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix2.png')

# Saving the Model

In [ ]:
model.save('food_vision_model.h5')

# Testing The Model

In [ ]:
def predict_image(img_path, model):
    # Load and preprocess the image
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    img = tf.expand_dims(img, axis=0)  # Add batch dimension

    # Predict
    predictions = model.predict(img)
    predicted_index = tf.argmax(predictions[0]).numpy()
    predicted_class = class_names[predicted_index]
    confidence = predictions[0][predicted_index]

    # Output
    print(f"Prediction: {predicted_class} ({confidence:.2%} confidence)")
    return predicted_class, confidence


In [ ]:
predict_image("/content/baklava.jpg", model)